!['Ex1'](img/ex1.png)

_Dados Fornecidos_

In [2]:
P = [75, 3*10**3, 3*10**3, 75] #kPa
T3 = 350 #C
x1 = 0
nbomba, nturb = 1, 1 

_Bibliotecas necessárias_

In [3]:
from CoolProp.CoolProp import State as st
import pandas as pd
import numpy as np

_Definição dos estados_

In [4]:
fluido = 'Water'

In [5]:
st1 = st(fluido, {'P': P[0], 'Q': x1})
st3 = st(fluido, {'P': P[2], 'T': T3 + 273.15})

_Estados com 100% de eficiência isentrópica (ciclo ideal)_

In [6]:
st2s = st(fluido, {'S': st1.s, 'P': P[1]})
st4s = st(fluido, {'S': st3.s, 'P': P[3]})

_Eficiência isentrópica_ <br/>
Turbina: $$ \eta_{turb} = \frac{h_{3} - h_{4}}{h_{3} - h_{4s}} $$
Bomba: $$ \eta_{pump} = \frac{h_{2s} - h_{1}}{h_{2} - h_{1}} $$

In [7]:
h2 = ((st2s.h-st1.h) / nbomba) + st1.h
h4 = st3.h - nturb*(st3.h - st4s.h)

In [8]:
st2 = st(fluido, {'H': h2, 'P': P[1]})
st4 = st(fluido, {'P': P[3], 'H': h4})

_Obtendo dados de temperatura, pressão, entalpia, entropia e titulo dos 4 estados_

In [15]:
temperatura, pressao, entalpia, entropia, titulo, estados = [], [], [], [], [], np.arange(1, 5, 1)

ets = st1, st2, st3, st4 

for prop in ets: 
    temperatura.append(prop.T)
    pressao.append(prop.p)
    entalpia.append(prop.h)
    entropia.append(prop.s)

    if np.absolute(prop.Q) == 1:
        titulo.append('Vap. Sat.')
    elif np.absolute(prop.Q) == 0:
        titulo.append('Liq. Sat.')
    else:
        titulo.append(np.absolute(prop.Q))

In [19]:
data_st = {
    'Estado': estados,
    'Temperatura [ºC]': temperatura,
    'Pressão [kPa]': pressao,
    'Entalpia [kJ/kg]': entalpia,
    'Entropia [kJ/kg/K]': entropia,
    'Titulo [-]': titulo
}

df_st = pd.DataFrame(data_st).style.hide(axis='index').format(precision=2)

In [20]:
display(df_st)

Estado,Temperatura [ºC],Pressão [kPa],Entalpia [kJ/kg],Entropia [kJ/kg/K],Titulo [-]
1,364.91,75.00,384.44,1.21,Liq. Sat.
2,365.09,3000.00,387.47,1.21,Vap. Sat.
3,623.15,3000.00,3116.06,6.74,Vap. Sat.
4,364.91,75.00,2403.02,6.74,0.89


$$ \omega_{pump, in} = h_2 - h_1 $$
$$ \omega_{turb, out} = h_3 - h_4 $$
$$ q_{in} = h_3 - h_2 $$
$$ q_{out} = h_4 - h_1 $$
$$\eta_{th} = 1 - \frac{q_{out}}{q_{in}}$$

In [59]:
w_pump_in = [st2.h - st1.h]
w_turb_out = [st3.h - st4.h]
q_in = [st3.h - st2.h]
q_out = [st4.h - st1.h]
n = [(1 - (q_out[0]/q_in[0]))*100]

In [60]:
data = {
    'w_pump_in [kJ/kg]': w_pump_in,
    'w_turb_out [kJ/kg]': w_turb_out,
    'q_in [kJ/kg]': q_in,
    'q_out [kJ/kg]': q_out,
    'η [%]': n
}

df = pd.DataFrame(data).style.hide(axis='index').format(precision=2)

In [61]:
display(df)

w_pump_in [kJ/kg],w_turb_out [kJ/kg],q_in [kJ/kg],q_out [kJ/kg],η [%]
3.03,713.04,2728.59,2018.58,26.02


_Exportando para o excel_

In [54]:
def save_xls(dict_df, path):

    writer = pd.ExcelWriter(path)
    for key in dict_df:
        dict_df[key].to_excel(writer, key, index=False)

    writer.save()

dict_df = {
    'Estados': df_st,
    'Resultados': df
}

path = r'res_ex1.xlsx'

save_xls(dict_df, path)